In [1]:
#comment out if using a local environment
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd "gdrive/My Drive/TopicModelling2/TopicModelling"

/content/gdrive/My Drive/TopicModelling2/TopicModelling


In [ ]:
#download spacy's french language model and pyLDAvis for Topic Modelling visualization
!python -m spacy download fr_core_news_sm
!pip install pyLDAvis

In [4]:
import pandas as pd
import numpy as np
import warnings
from collections import Counter

from utilsTM import *

from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases
from gensim.models.ldamodel import LdaModel, CoherenceModel
from gensim.corpora.dictionary import Dictionary

import pyLDAvis
import pyLDAvis.gensim #this is needed

import spacy

#import nltk

### Aim Of The Project
The aim of this notebook is to perform topic modelling on a corpus of company descriptions extracted from the [Commercial Register of Geneva](https://www.ge.ch/recherche-entreprises-registre-du-commerce-geneve).  
This could be used to analyze the economic landscape of the city and to build similarities from the topics embeddings.  
*(Please refer to the commercial register of Geneva for information on restrictions of use of the data)*

### Preprocessing the corpus
To get meaningful topics we need to remove words which are not relevant to categorize company descriptions.  
The steps are the following (the order can be changed):


1.   Remove punctations, lower case, accents ...
2.   Remove links, frequent unuseful information
3.   Tokenize
4.   Remove stop words, named entities
5.   Remove words other than noun, adjectif, verb or adverb
6.   Lemmatize
7.   Build bi-grams and tri-grams
8.   Remove frequent words, list of manually chosen words, verbs 

Not that the last step is specific to this task.



In [5]:
#remove deprecation warning messages from ipywidgets
warnings.filterwarnings(action="ignore")

#TRY LDA FROM PYSPARK MAYBE YOU CAN GET BETTER RESULTS...

Preprocessing step functions

In [6]:
# a list of processing steps which can be used as in different order in a processing pipeline, these functions are partly reusable.

def removeCfs(corpus):
  '''a lot of doc contain (cf. statuts pour but complet), (cf. but de l'établissement principal), ... at the end of the description. We remove them.'''
  corpus = [doc[:doc.find("(cf")] for doc in corpus]
  return corpus

def gensimPreprocess(corpus):
  '''perform a number of preliminary preprocessing steps using gensim's simple_preprocess() (remove punctations, lowercase etc...).'''
  corpus = [simple_preprocess(doc,deacc=False,max_len=30) for doc in corpus]
  return corpus

def spacyPreprocess(corpus):
  ''''Remove stopwords, punctation, named entities, words that are not a noun, adjectif, verb or adverb, lemmatize.'''
  nlp = spacy.load("fr_core_news_sm") #load a spaCy french language model
  keepPos = ['NOUN', 'ADJ', 'VERB', 'ADV']
  corpusNew = []
  for doc in deTokenize(corpus):
    doc = nlp(doc) #perform text analysis on "doc" with the model from spacy.
    doc = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop and (token.pos_ in keepPos) and not token.ent_type]
    corpusNew.append(doc)
  
  return corpusNew

def removeVerbs(corpus):
  '''Remove french verbs, spacy POS for verbs was not good, so we had to scrape a full list of french verbs (refer to the notebook ScrapingAllFrenchVerbs.ipynb).'''
  with open(file="frenchVerbs.txt",mode="r") as f:
    FRENCHVERBS = list(f.readlines())
    FRENCHVERBS = [verb.replace("\n","").strip() for verb in FRENCHVERBS]
  corpus = [[word for word in doc if word not in FRENCHVERBS] for doc in corpus]
  return corpus

def constructBigrams(corpus):
  '''Extract frequent set of two words.'''
  corpus = list(Phrases(sentences=corpus,min_count=2)[corpus])
  return corpus

def removeAccents(corpus):
  corpus = [simple_preprocess(doc,deacc=True,max_len=30) for doc in deTokenize(corpus)]
  return corpus
  
def removeNonDiscriminative(corpus):
  '''Remove word, which are present in most of the documents and are noise from a domain specific vocabulary (business) rather than discriminative features.'''
  #Take the most frequent word, manually set the threshold for the most common word number and keep some words which might be due to a frequent topic
  cnt = Counter([word for doc in corpus for word in doc])
  keep = ["financier","immobilier","courtage","batiment","materiel","construction","importation_exportation",
        "informatique","bien_immobilier","investissement","industriel","public","prohiber_lfaie","financement"]
  #remove 63 most common
  remove = [k for k,_ in cnt.most_common(63) if k not in keep]
  manual = ["but","pays","genre","aupres","groupe","lien","principalement","commercer","rapport","edition","geneve","premier","tiers"] #a list of unwanted words
  remove = remove + manual

  #remove words
  corpus = [[word for word in doc if word not in remove] for doc in corpus]
  
  return corpus

def removeEmpty(corpus):
  return [doc for doc in corpus if doc]

Preprocessing

In [7]:
#read the data from a csv file. The data was extracted from the commercial register of Geneva, and consists on company descriptions in French.
corpus0 = pd.read_csv("CommercialRegisterData.csv").Objet.to_list()

#For this first experiment we consider only 10'000 descriptions, but the register contains up to 50'000 companies.
print("Corpus size :",len(corpus0))

Corpus size : 9996


In [8]:
# we will use a small UI to vizualize the corpus more easily, as we go along the different preprocessing steps
Out0 = Output()
corpusVisualizer(corpus0,Out0)

In [9]:
#the pipeline of preprocessing steps
preprocessingSteps = [removeCfs, gensimPreprocess, removeVerbs,spacyPreprocess, removeAccents, constructBigrams, removeNonDiscriminative, removeEmpty]

#Probably inefficient for large datasets
corpuses = [corpus0]
for step in preprocessingSteps:
  corpuses.append(step(corpuses[-1]))

In [10]:
corpusFinal = corpuses[-1]

In [13]:
Out = Output()
compareCorpuses(corpus0,corpusFinal,Out)

### Fitting the model

In [14]:
id2word = Dictionary(documents=corpusFinal)

In [15]:
corpusBow = [id2word.doc2bow(doc) for doc in corpusFinal]

In [16]:
ntopics = 15
rs = 100
model = LdaModel(corpus=corpusBow,num_topics=ntopics,id2word=id2word,chunksize=2000,
                 passes=20,minimum_probability=0.0001,random_state=rs,per_word_topics=False)
coherence = CoherenceModel(model=model,texts=corpusFinal,dictionary=id2word).get_coherence()
print("Average correlation between multi-words in topics",round(coherence,4))

Average correlation between multi-words in topics 0.4542


In [17]:
#visualize the topics with pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpusBow, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.324342  0.089839       1        1  11.347607
0     -0.265357 -0.048700       2        1  11.312072
2     -0.014727  0.124625       3        1   9.294597
8      0.042132 -0.150125       4        1   7.551963
12     0.077524 -0.198755       5        1   6.840929
13     0.050933 -0.085858       6        1   6.649752
3     -0.104116  0.068952       7        1   6.318906
11    -0.011855 -0.028583       8        1   6.028956
4      0.131046  0.216596       9        1   5.747155
6      0.040228  0.054248      10        1   5.214843
5      0.137624  0.122289      11        1   5.097642
10     0.080699 -0.055825      12        1   5.060285
9      0.004709 -0.122287      13        1   4.798237
7      0.107227  0.136156      14        1   4.752790
14     0.048276 -0.122571      15        1   3.984264, topic_info=              Term         Freq        Total Category  logprob  loglift
11      immobilier  1104.000000  1104.000000  Default  30.0000  30.0000
99       financier  1183.000000  1183.000000  Default  29.0000  29.0000
202   informatique   419.000000   419.000000  Default  28.0000  28.0000
126       batiment   350.000000   350.000000  Default  27.0000  27.0000
6         courtage   433.000000   433.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
1073        suisse    27.139027    83.195409  Topic15  -4.7223   2.1026
407         social    34.439272   164.385905  Topic15  -4.4841   1.6598
69          action    24.125069    71.778051  Topic15  -4.8400   2.1325
284         public    21.337012   166.156939  Topic15  -4.9629   1.1703
1443         depot    16.453683    36.663286  Topic15  -5.2228   2.4216

[796 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1041      3  0.130422  accessoire
1041      4  0.176701  accessoire
1041      9  0.214565  accessoire
1041     10  0.122008  accessoire
1041     11  0.117801  accessoire
...     ...       ...         ...
6563      1  0.939646    viticole
573      14  0.984606     voiture
525       5  0.921495         vol
1234     10  0.753921      voyage
1234     13  0.230365      voyage

[1261 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 9, 13, 14, 4, 12, 5, 7, 6, 11, 10, 8, 15])

### Building a similarity matrix to perform queries
(Next step)

In [ ]:
from gensim import similarities
index = similarities.MatrixSimilarity(model[corpusBow])